In [19]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import vuong_tests2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


yn,xn,nobs = gen_data()
print(xn.shape)
print(sm.add_constant(xn).shape)
print(scipy.stats.mode(yn))

(1000,)
(1000, 2)
ModeResult(mode=array([0.]), count=array([81]))


In [21]:
class Tobit(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(Tobit,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        
        beta = params
        
        mu_y = np.matmul(x,beta)
        
        pr_y = stats.norm.logpdf( y, loc = mu_y)
        
       
        #if complete case, assign pr missing to all observations...
        pr_m = stats.norm.logcdf( y, loc = mu_y)
        
        #we're done if ols
        if self.ols:
            return pr_y
        else:
            ll = (1-m)*pr_y + m*pr_m
            return ll
        
    def score(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        m_x = np.repeat(m,x.shape[1]).reshape(x.shape)
        
        if ols: #if OLS use all the data...
            m, m_x = np.ones(y.shape), np.ones(x.shape)
        
        
        b = params

        beta_jac = np.zeros(len(b))
        
        #for censored
        if not ols: 
            left_stats = (y - np.dot(x, b)) 
            l_pdf = scipy.stats.norm.logpdf(left_stats)
            l_cdf = scipy.stats.norm.logcdf(left_stats)
            left_frac = np.exp(l_pdf - l_cdf)
            beta_left = np.dot(left_frac*m, x*m_x)
            beta_jac -= beta_left
        
        #for non-censored
        mid_stats = (y - np.dot(x, b))
        beta_mid = np.dot(mid_stats*(1-m), x*(1-m_x) )
        beta_jac += beta_mid
        
        # by chain rule, since the expression above is dloglik/dlogsigma
        return beta_jac



model1 =  Tobit(yn,sm.add_constant(xn))
model1_fit = model1.fit(disp=False)
print(model1_fit.summary())

model2 =  Tobit(yn,sm.add_constant(xn),ols=True)
model2_fit = model2.fit(disp=False)
print(model2_fit.summary())

                                Tobit Results                                 
Dep. Variable:                      y   Log-Likelihood:                -1346.0
Model:                          Tobit   AIC:                             2696.
Method:            Maximum Likelihood   BIC:                             2706.
Date:                Fri, 26 Mar 2021                                         
Time:                        18:09:40                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9520      0.047     20.168      0.000       0.860       1.045
x1             0.2577      0.007     38.003      0.0

In [22]:
def setup_shi(yn,xn):
    model1 = Tobit(yn,sm.add_constant(xn))
    model1_fit = model1.fit(disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score_obs(model1_fit.params)    
    hess1 = model1.hessian(model1_fit.params)
    k1 = model1_fit.params
    
    #fit logistic values
    model2 = Tobit(yn,sm.add_constant(xn),ols=True)
    model2_fit = model2.fit(disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score_obs(model2_fit.params)    
    hess2 = model2.hessian(model2_fit.params)
    k2 = model2_fit.params
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

In [23]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.uniform.rvs(loc=-5,scale=10,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs

shi_result = vuong_tests2.monte_carlo_shi(1,setup_shi,gen_data)
print(shi_result)

reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

[1. 0. 0.]
reg: [0. 0. 1.], boot1: [0. 0. 1.], boot2: [0. 0. 1.], llr:-29.43610634171672, std: 5.02742704891183, omega:0.18942375395835506


In [24]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.75 0.   0.25], boot1: [0.8 0.  0.2], boot2: [0.75 0.   0.25], llr:-5.012767885040398, std: 5.754050044131698, omega:0.167933448964015
[1. 0. 0.]


In [25]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 250
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.89 0.   0.11], boot1: [0.87 0.01 0.12], boot2: [0.87 0.01 0.12], llr:-0.7866424771225516, std: 2.8242281472528368, omega:0.17472776029090753
[1. 0. 0.]


# Main examples

In [26]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.87 0.01 0.12], boot1: [0.88 0.01 0.11], boot2: [0.86 0.01 0.13], llr:-2.4470144033068792, std: 4.078352409772033, omega:0.1682114449843301
[1. 0. 0.]


In [27]:
beta0 = 1.
beta1 = .5

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:44.72371216030277, std: 11.083246736507375, omega:0.5280182162493349
[0.13 0.87 0.  ]


In [28]:
beta0 = 1.
beta1 = 1.

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:220.75407502695867, std: 46.187186203191075, omega:2.026734139817817
[0.26 0.74 0.  ]


# Misc examples

In [29]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs

reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:446.7049754938491, std: 62.12020961706232, omega:2.026514472977945
[0.05 0.95 0.  ]


In [30]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 0. 1.], boot1: [0. 0. 1.], boot2: [0. 0. 1.], llr:-172.8930473174962, std: 19.30239474075414, omega:1.005766944669856
[0.15 0.   0.85]


In [31]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 2
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 0. 1.], boot1: [0. 0. 1.], boot2: [0. 0. 1.], llr:-106.7650665605856, std: 8.647186532290823, omega:0.28901176399794165
[0. 0. 1.]


In [32]:
def gen_data(beta0=3.29543322,beta1=0):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

../vuong_tests2.py:78: RuntimeWarning: invalid value encountered in double_scalars
  test_stats_naive.append(llrb/ np.sqrt( (ll1b -ll2b).var()*nobs) )
../vuong_tests2.py:46: RuntimeWarning: invalid value encountered in double_scalars
  test_stat = llr/(omega*np.sqrt(nobs))


IndexError: index 3 is out of bounds for axis 0 with size 3